In [ ]:
!pip install torch torchvision
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
import tqdm
import numpy as np
import os
import json

In [ ]:
device = 'cuda'
model, preprocess = clip.load('ViT-B/32', device=device)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 285MiB/s]


In [ ]:
test_strings = [['a brunette person', 'a blonde person'],
                ['an young person', 'an old person'],
                ['brunette', 'blonde'],
                ['old', 'young'],
                ['weird', 'normal'],
                ['orange', 'blue'],
                ['light', 'dark'],
                ['happy', 'sad']]
labels = ['b2b-person', 'y2o-person', 'b2b', 'y2o', 'w2n', 'o2b', 'l2d', 'h2s']
n = len(test_strings)

In [ ]:
folder = 'data/b2b_final'
paths = [os.path.join(folder, file) for file in os.listdir(folder)]
paths = [path for path in paths if path.endswith('.png')]

In [ ]:
m = len(paths)

In [ ]:
m

26

In [ ]:
result_lists = []
for path in tqdm.tqdm(paths):
  image = preprocess(Image.open(path)).unsqueeze(0).to(device)
  results = []

  for binary in test_strings:
    text = clip.tokenize(binary).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

        results.append(probs.tolist()[0])
  result_lists.append(results)

100%|██████████| 26/26 [00:09<00:00,  2.77it/s]


In [ ]:
averages = []
for i in range(n):
  sum = 0
  for j in range(m):
    sum += result_lists[j][i][1]
  averages.append(sum / m)

In [ ]:
averages

[0.6023711424607497,
 0.1309070587158203,
 0.7629327774047852,
 0.5301889272836539,
 0.5477576622596154,
 0.45837637094350964,
 0.6165020282451923,
 0.4525105403019832]

In [ ]:
for i in range(len(paths)):
  path = paths[i]
  with open(path[:-4] + '_CLIP.json', 'w') as f:
    json.dump({labels[j]:result_lists[i][j] for j in range(n)}, f, indent=2)